Part 3a: Chain of Thought (CoT)

2. Concept: Latent Reasoning

Name: Vaidurya Samaga

SRN: PES2UG23CS668

Section:K

In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.1 MB/s eta 0:00:00


In [3]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b (Small/Fast) to demonstrate logic failures
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Enter your Groq API Key: ··········


3. The Experiment: A Tricky Math Problem

In [4]:
question = "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many does he have now?"

# 1. Standard Prompt (Direct Answer)
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to add the initial number of tennis balls he had (5) to the number of tennis balls he bought (2 cans * 3 tennis balls per can).

2 cans * 3 tennis balls per can = 6 tennis balls

Now, let's add the initial number of tennis balls (5) to the number of tennis balls he bought (6):

5 + 6 = 11

So, Roger now has 11 tennis balls.


In [5]:
prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find out how many tennis balls Roger has now, we need to follow these steps:

1. Roger already has 5 tennis balls.
2. He buys 2 more cans of tennis balls. Each can has 3 tennis balls, so he buys 2 x 3 = 6 more tennis balls.
3. Now, we add the tennis balls he already had (5) to the new tennis balls he bought (6). 5 + 6 = 11

So, Roger now has 11 tennis balls.


Part 3b: Tree of Thoughts (ToT) & Graph of Thoughts (GoT)

In [6]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed


In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can I get my 5-year-old to eat vegetables?"

# Step 1: The Branch Generator
prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give me one unique, creative solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

# Step 2: The Judge
prompt_judge = ChatPromptTemplate.from_template(
    """
    I have three proposed solutions for: '{problem}'

    1: {sol1}
    2: {sol2}
    3: {sol3}

    Act as a Child Psychologist. Pick the most sustainable one (not bribery) and explain why.
    """
)

# Chain: Input -> Branches -> Judge -> Output
tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))

--- Tree of Thoughts (ToT) Result ---
As a Child Psychologist, I would recommend **Solution 3: Get Them Involved in the Process** as the most sustainable approach to encouraging a 5-year-old to eat vegetables. This approach aligns with the principles of child-centered learning and development, which prioritize autonomy, agency, and engagement.

Here's why I believe this approach is the most sustainable:

1. **Motivation through involvement**: When children are involved in the process of meal planning, grocery shopping, and cooking, they feel more invested in the outcome. This sense of ownership and agency can motivate them to try new foods, including vegetables.
2. **Learning and exploration**: By involving children in the cooking process, they can learn about the different textures, smells, and tastes of vegetables. This hands-on experience can foster a sense of curiosity and exploration, making vegetables more appealing and interesting.
3. **Development of fine motor skills**: Activi

In [8]:
# 1. The Generator (Divergence)
prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence movie plot about: {topic}. Genre: {genre}."
)

drafts = RunnableParallel(
    draft_scifi=prompt_draft.partial(genre="Sci-Fi") | llm | StrOutputParser(),
    draft_romance=prompt_draft.partial(genre="Romance") | llm | StrOutputParser(),
    draft_horror=prompt_draft.partial(genre="Horror") | llm | StrOutputParser(),
)

# 2. The Aggregator (Convergence)
prompt_combine = ChatPromptTemplate.from_template(
    """
    I have three movie ideas for the topic '{topic}':
    1. Sci-Fi: {draft_scifi}
    2. Romance: {draft_romance}
    3. Horror: {draft_horror}

    Your task: Create a new Mega-Movie that combines the TECHNOLOGY of Sci-Fi, the PASSION of Romance, and the FEAR of Horror.
    Write one paragraph.
    """
)

# 3. The Chain
got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Time Travel"))

--- Graph of Thoughts (GoT) Result ---
Here's a mega-movie concept that combines the technology of sci-fi, the passion of romance, and the fear of horror:

"Timeless Requiem" is a heart-pounding, mind-bending thriller that follows Dr. Emma Taylor, a brilliant physicist who has spent her entire career developing a revolutionary time-travel device. When she finally succeeds in building a functioning time machine, she finds herself transported back to the 19th century, where she meets a charming and enigmatic artist named Julian. As Emma and Julian's passion for each other grows, they must navigate the treacherous landscape of Victorian-era London, where a notorious serial killer known as the "Shadow Weaver" is stalking and murdering victims, leaving behind a trail of eerie supernatural occurrences that suggest a dark and ancient evil is at work. But as Emma tries to escape the clutches of the Shadow Weaver, she realizes that her time-travel device has created a catastrophic ripple effect

| Method                     | Structure                                      | Best For                                                 | Cost / Latency |
| -------------------------- | ---------------------------------------------- | -------------------------------------------------------- | -------------- |
| **Simple Prompting**       | Input → Output                                 | Simple facts, direct answers, summaries                  | Low            |
| **CoT (Chain of Thought)** | Input → Steps → Output                         | Mathematical reasoning, logic, debugging                 | Medium         |
| **ToT (Tree of Thought)**  | Input → Multiple Branches → Selection → Output | Planning, decision making, brainstorming                 | High           |
| **GoT (Graph of Thought)** | Input → Branch → Merge / Aggregate → Output    | Research synthesis, creative reasoning, complex analysis | Very High      |
